In [3]:
!pip install --quiet duckdb
!pip install --quiet jupysql
!pip install --quiet duckdb-engine

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.0/95.0 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 192.8/192.8 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 1.9 MB/s eta 0:00:00


In [4]:
import duckdb
import pandas as pd

# Import jupysql Jupyter extension to create SQL cells
%load_ext sql

In [5]:
%sql duckdb:///duckdb.db

Connecting to 'duckdb:///duckdb.db'

In [6]:
%%sql

CREATE SCHEMA IF NOT EXISTS raw_data;
CREATE SCHEMA IF NOT EXISTS analytics;

Running query in 'duckdb:///duckdb.db'

Count


In [7]:
%%sql

DROP TABLE IF EXISTS raw_data.user_session_channel;
CREATE TABLE IF NOT EXISTS raw_data.user_session_channel (
    userid int,
    sessionid varchar(32) primary key,
    channel varchar(32)
);

DROP TABLE IF EXISTS raw_data.session_timestamp;
CREATE TABLE IF NOT EXISTS raw_data.session_timestamp (
    sessionid varchar(32) primary key,
    ts TIMESTAMP
);

DROP TABLE IF EXISTS raw_data.session_transaction;
CREATE TABLE IF NOT EXISTS raw_data.session_transaction (
    sessionid varchar(32) primary key,
    refunded boolean,
    amount integer
);

Running query in 'duckdb:///duckdb.db'

Count


In [8]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/user_session_channel.csv
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/session_timestamp.csv
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/session_transaction.csv

--2024-10-01 03:56:29--  https://s3-geospatial.s3.us-west-2.amazonaws.com/user_session_channel.csv
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 52.218.232.89, 52.92.137.210, 52.92.227.242, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)|52.218.232.89|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5037609 (4.8M) [text/csv]
Saving to: ‘user_session_channel.csv’

user_session_channe 100%[===================>]   4.80M  8.54MB/s    in 0.6s    

2024-10-01 03:56:30 (8.54 MB/s) - ‘user_session_channel.csv’ saved [5037609/5037609]

--2024-10-01 03:56:30--  https://s3-geospatial.s3.us-west-2.amazonaws.com/session_timestamp.csv
Resolving s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.com)... 52.218.232.89, 52.92.137.210, 52.92.227.242, ...
Connecting to s3-geospatial.s3.us-west-2.amazonaws.com (s3-geospatial.s3.us-west-2.amazonaws.co

In [9]:
!ls -tl

total 10908
-rw-r--r-- 1 root root     731 Oct  1 03:56 duckdb.db.wal
-rw-r--r-- 1 root root   12288 Oct  1 03:56 duckdb.db
drwxr-xr-x 1 root root    4096 Sep 25 18:24 sample_data
-rw-r--r-- 1 root root   44041 May  4 15:58 session_transaction.csv
-rw-r--r-- 1 root root 5037609 May  4 15:58 user_session_channel.csv
-rw-r--r-- 1 root root 6064988 May  4 15:58 session_timestamp.csv


In [10]:
%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False

In [11]:
%%sql

INSERT INTO raw_data.user_session_channel SELECT * FROM 'user_session_channel.csv';
INSERT INTO raw_data.session_timestamp SELECT * FROM 'session_timestamp.csv';
INSERT INTO raw_data.session_transaction SELECT * FROM 'session_transaction.csv';

,Success


In [12]:
%%sql

SELECT COUNT(1) FROM raw_data.user_session_channel;

,count(1)
0,101520


In [13]:
%%sql

SELECT COUNT(1) FROM raw_data.session_timestamp;

,count(1)
0,101520


In [14]:
%%sql

SELECT COUNT(1) FROM raw_data.session_transaction

,count(1)
0,996


In [15]:
%%sql

SELECT *
FROM raw_data.User_session_channel
LIMIT 10;

,userid,sessionid,channel
0,184,c41dd99a69df04044aa4e33ece9c9249,Naver
1,80,fdc0eb412a84fa549afe68373d9087e9,Organic
2,251,0a54b19a13b6712dc04d1b49215423d8,Facebook
3,264,a914ecef9c12ffdb9bede64bb703d877,Google
4,744,05ae14d7ae387b93370d142d82220f1b,Facebook
5,229,eddeb82df22554fa67c641e3f8a25566,Google
6,265,4c4ea5258ef3fb3fb1fc48fee9b4408c,Naver
7,80,8ae1da0fe37c98412768453f82490da2,Organic
8,45,60131a2a3f223dc8f4753bcc5771660c,Youtube
9,198,90f4760fcc9b69c13da7368c5c2917f3,Facebook


## Use duckDB API

In [17]:
duckdb_con = duckdb.connect("duckdb.db")

ConnectionException: Connection Error: Can't open a connection to same database file with a different configuration than existing connections

In [ ]:
df = duckdb_con.execute('SELECT * FROM raw_data.user_session_channel').df()

In [ ]:
df.head()

## DDL 실습

Table 생성 (PRIMARY KEY, NULL 속성, default 값)

In [ ]:
%%sql

CREATE SCHEMA IF NOT EXISTS adhoc;

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.test;
CREATE TABLE adhoc.test
(
    id int PRIMARY KEY,
    value int NOT NULL, -- NOT NULL이 지정안되면 기본으로 NULL값 지정 가능
    location varchar(256) default 'seoul'
);

In [ ]:
%%sql

-- 기본기 유일성 체크
INSERT INTO adhoc.test VALUES (1, 2, 'busan'), (1, 3, 'san jose');

In [ ]:
%%sql

SELECT * FROM adhoc.test;

In [ ]:
%%sql

-- 기본값 사용
INSERT INTO adhoc.test (id, value) VALUES (1, 100);

In [ ]:
%%sql

-- NULL 값 허용 여부가 어떻게 적용되는지?
INSERT INTO adhoc.test VALUES (4, NULL, 'LA');

In [ ]:
%%sql

-- Primary key로 NULL을 지정하면?
INSERT INTO adhoc.test VALUES (NULL, 3, 'LA');

In [ ]:
%%sql

INSERT INTO adhoc.test VALUES (3, 4, NULL);

CTAS 실습


In [ ]:
%%sql

-- 먼저 SELECT 실행해보기
SELECT DISTINCT channel
FROM raw_data.user_session_channel;

In [ ]:
%%sql

-- CTAS로 테이블 만들기
DROP TABLE IF EXISTS analytics.channel;
CREATE TABLE analytics.channel AS
SELECT DISTINCT channel
FROM raw_data.user_session_channel;

In [ ]:
%%sql

SELECT *
FROM analytics.channel

ALTER TABLE 실습

In [ ]:
%%sql

-- 테이블 이름 바꾸기
ALTER TABLE analytics.channel RENAME TO channel_list;

In [ ]:
%%sql

SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'analytics';

In [ ]:
%%sql

-- 새로운 컬럼 추가하기
ALTER TABLE analytics.channel_list ADD COLUMN description varchar(512);

In [ ]:
%%sql

PRAGMA table_info('analytics.channel_list');

In [ ]:
%%sql

SELECT column_name, data_type, is_nullable, column_default
FROM information_schema.columns
WHERE table_schema = 'analytics' AND table_name = 'channel_list';

In [ ]:
%%sql

-- 컬럼 이름 바꾸기
ALTER TABLE analytics.channel_list RENAME COLUMN channel to name;

In [ ]:
%%sql

SELECT column_name, data_type, is_nullable, column_default
FROM information_schema.columns
WHERE table_schema = 'analytics' AND table_name = 'channel_list';

In [ ]:
%%sql

-- 컬럼 삭제하기
ALTER TABLE analytics.channel_list DROP COLUMN description;

Table 삭제하기

In [ ]:
%%sql

-- 레코드 삭제하기
DELETE FROM analytics.channel_list; -- WHERE로 삭제 조건을 걸 수 있음

In [ ]:
%%sql

SELECT *
FROM analytics.channel_list;

In [ ]:
%%sql

TRUNCATE analytics.channel_list;

In [ ]:
%%sql

SELECT table_name
FROM information_schema.tables
WHERE table_schema = 'analytics';

In [ ]:
%%sql

-- 테이블 삭제하기
DROP TABLE IF EXISTS analytics.channel_list;

## DML 소개

In [ ]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/nps.csv

In [ ]:
%%sql

DROP TABLE IF EXISTS raw_data.nps;
CREATE TABLE raw_data.nps
(
    id int PRIMARY KEY,
    created TIMESTAMP NOT NULL,
    score SMALLINT NOT NULL
);

In [ ]:
%%sql

COPY raw_data.nps FROM 'nps.csv' (DELIMITER ',');
-- INSERT INTO raw_data.nps SELECT * FROM 'nps.csv';

In [ ]:
%%sql

SELECT *
FROM raw_data.nps
LIMIT 10;

In [ ]:
%%sql

SELECT *
FROM raw_data.nps
WHERE id = 1;

In [ ]:
%%sql

UPDATE raw_data.nps
SET score = 8
WHERE id = 1;

In [ ]:
%%sql

SELECT *
FROM raw_data.nps
WHERE id = 1;

In [ ]:
%%sql

SELECT score, COUNT(1)
FROM raw_data.nps
GROUP BY score; -- GROUP BY 1;

In [ ]:
%%sql

DELETE FROM raw_data.nps
WHERE score = 10;

In [ ]:
%%sql

SELECT score, COUNT(1)
FROM raw_data.nps
GROUP BY 1;

## SQL SELECT 실습

In [ ]:
%config SqlMagic.autopandas = False

In [ ]:
%%sql

SELECT *
FROM raw_data.user_session_channel
LIMIT 10;

In [ ]:
%%sql

SELECT  userid, sessionid, channel
FROM raw_data.user_session_channel
LIMIT 10;

In [ ]:
%%sql

-- 유일한 채널 이름을 알고 싶은 경우
SELECT DISTINCT channel
FROM raw_data.user_session_channel;

In [ ]:
%%sql

-- 레코드수를 카운트하고 싶은 경우. COUNT 함수!!
-- COUNT 함수는 어떻게 동작하는가? COUNT(100), COUNT(NULL)
SELECT COUNT(1)
FROM raw_data.user_session_channel;

In [ ]:
%%sql

SELECT COUNT(1), COUNT(0), COUNT(100), COUNT(NULL)
FROM raw_data.user_session_channel;

In [ ]:
%%sql

-- channel 이름이 Facebook경우만 레코드수 카운트
SELECT COUNT(1)
FROM raw_data.user_session_channel
WHERE channel = 'Facebook';

In [ ]:
%%sql

SELECT NULL+1, NULL*1, NULL-1, NULL/2;

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.count_test;
CREATE TABLE adhoc.count_test (
  value int
);
INSERT INTO adhoc.count_test VALUES (NULL),(1), (1), (0), (0), (4), (3);

In [ ]:
%%sql

SELECT *
FROM adhoc.count_test;

In [ ]:
%%sql

SELECT
    COUNT(1),
    COUNT(0),
    COUNT(NULL),
    COUNT(value),
    COUNT(DISTINCT value)
FROM adhoc.count_test;

In [ ]:
%%sql


-- channel이 Facebook이나 Instagram이면 Social-Media로 분류
-- channel이 Google, Naver라면 Search-Engine으로 분류
-- 그 외에는 Something-Else로 분류
SELECT
  channel,
  CASE
    WHEN channel in ('Facebook', 'Instagram') THEN 'Social-Media'
    WHEN channel in ('Google', 'Naver') THEN 'Search-Engine'
    ELSE 'Something-Else'
  END channel_type
FROM raw_data.user_session_channel
LIMIT 10;

In [ ]:
%%sql

SELECT COUNT(1)
FROM raw_data.user_session_channel
WHERE channel in ('Google','Facebook');

In [ ]:
%%sql

-- ILIKE은 대소문자를 구별하지 않은 문자열 비교. LIKE은 구별하는 비교
SELECT COUNT(1)
FROM raw_data.user_session_channel
WHERE channel ILIKE 'Google' or channel ILIKE 'Facebook';

In [ ]:
%%sql

SELECT DISTINCT channel
FROM raw_data.user_session_channel
WHERE channel ILIKE '%o%';

In [ ]:
%%sql

SELECT DISTINCT channel
FROM raw_data.user_session_channel
WHERE channel NOT ILIKE '%o%';

In [ ]:
%%sql

SELECT DISTINCT
    channel,
    LEN(channel) len,
    UPPER(channel) upper,
    LOWER(channel) lower,
    LEFT(channel, 4) left4,
    SUBSTRING(channel, 2, 2) subs,
    RPAD(channel, 10, '*') rpadded_channel,
    LPAD(channel, 10, '*') lpadded_channel
FROM raw_data.user_session_channel
LIMIT 10;

Text Functions: https://duckdb.org/docs/sql/functions/char.html

In [ ]:
%%sql

SELECT *
FROM adhoc.count_test
ORDER BY value DESC;

In [ ]:
%%sql

SELECT *
FROM adhoc.count_test
ORDER BY value ASC;

In [ ]:
%%sql

SELECT *
FROM adhoc.count_test
ORDER BY value ASC NULLS FIRST;

In [ ]:
%%sql

SELECT
    CURRENT_DATE date,
    CURRENT_TIMESTAMP ts,
    DATE_PART('year', CURRENT_DATE) y,
    DATE_PART('month', CURRENT_DATE) m,
    DATE_PART('day', CURRENT_DATE) d,
    DATE_TRUNC('year', CURRENT_DATE) yt, -- 날짜 형태로 리턴 "-01-01"을 추가
    DATE_TRUNC('month', CURRENT_DATE) mt, -- 날짜 형태로 리턴 "-01"을 추가
    DATE_TRUNC('day', CURRENT_DATE) dt;

In [ ]:
%%sql

-- 두 번째 파라미터가 가리키는 시간은 첫번째 인자가 가리키는 시간대로 이를 GMT로 변환
SELECT TIMEZONE('America/Denver', TIMESTAMP '2024-07-01 23:38:40') AS gmt_time

In [ ]:
%%sql

-- 두 번째 파라미터가 가리키는 시간은 동부 시간임 (-05).
-- 이를 첫번째 인자가 가리키는 시간으로 변환 (마운틴 시간). 동부 시간에서 두 시간을 빼게 됨
SELECT TIMEZONE('America/Denver', TIMESTAMPTZ '2001-02-16 20:38:40-05') AS gmt_time

In [ ]:
%%sql

SELECT (1/2)::int, cast((1/2) as varchar), 100::varchar;

##

In [ ]:
%%sql

SELECT
    STRFTIME(CURRENT_TIMESTAMP, '%Y-%m-%d') AS ymd,               -- 타임스탬프를 문자열로 변환
    STRPTIME('2024-07-09 12:34:56', '%Y-%m-%d %H:%M:%S') dt;  -- 문자열을 타임스탬프 타입으로 변환

### Complex Type

In [ ]:
%%sql

-- LIST는 가변 길이를 갖는 배열
DROP TABLE IF EXISTS adhoc.students;
CREATE TABLE adhoc.students (
    id INTEGER,
    name VARCHAR,
    scores INTEGER[]
);

-- Insert data
INSERT INTO adhoc.students VALUES
    (1, 'Alice', [85, 90, 78]),
    (2, 'Bob', [92, 88, 95, 100]);

In [ ]:
%%sql

SELECT *
FROM adhoc.students;

In [ ]:
%%sql

-- Query the LIST
SELECT name, scores[2] AS second_score
FROM adhoc.students;

In [ ]:
%%sql

-- Unnest the LIST
SELECT name, unnest(scores) AS individual_score
FROM adhoc.students;

In [ ]:
%%sql

-- ARRAY는 고정된 길이를 갖는 배열
DROP TABLE IF EXISTS adhoc.sensors;
CREATE TABLE adhoc.sensors (
    id INTEGER,
    readings DOUBLE[3]
);

-- Insert data
INSERT INTO adhoc.sensors VALUES
    (1, [10.5, 11.2, 9.8]),
    (2, [22.1, 21.9, 22.3]);

In [ ]:
%%sql

SELECT *
FROM adhoc.sensors;

In [ ]:
%%sql

-- Query the ARRAY
SELECT id, readings[1] AS first_reading
FROM adhoc.sensors;

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.employees;
-- Create a table with a STRUCT column
CREATE TABLE adhoc.employees (
    id INTEGER,
    info STRUCT(name VARCHAR, age INTEGER, department VARCHAR)
);

-- Insert data
INSERT INTO adhoc.employees VALUES
    (1, {'name': 'Alice', 'age': 30, 'department': 'IT'}),
    (2, {'name': 'Bob', 'age': 35, 'department': 'HR'});

In [ ]:
%%sql

SELECT *
FROM adhoc.employees;

In [ ]:
%%sql

-- Query the STRUCT
SELECT id, info.name, info.age
FROM adhoc.employees
WHERE info.department = 'IT';

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.product_attributes;
-- Create a table with a MAP column
CREATE TABLE adhoc.product_attributes (
    id INTEGER,
    attributes MAP(VARCHAR, VARCHAR)
);

-- Insert data
INSERT INTO adhoc.product_attributes VALUES
    (1, MAP(['color', 'size'], ['red', 'large'])),
    (2, MAP(['color', 'material'], ['blue', 'cotton']));

In [ ]:
%%sql

SELECT *
FROM adhoc.product_attributes;

In [ ]:
%%sql

-- Query the MAP
SELECT id, attributes['color'] AS color
FROM adhoc.product_attributes;

In [ ]:
%%sql

-- Unnest the MAP
SELECT
    id,
    unnest(map_keys(attributes)) AS attribute,
    unnest(map_values(attributes)) AS value
FROM adhoc.product_attributes;

In [ ]:
%%sql

-- Create a table with a UNION column
DROP TABLE IF EXISTS adhoc.product_attributes;
-- 제품 속성 테이블 생성
CREATE TABLE adhoc.product_attributes (
    product_id INTEGER,
    attribute_name VARCHAR,
    attribute_value UNION(i INTEGER, f FLOAT, str VARCHAR, b BOOLEAN),
    PRIMARY KEY (product_id, attribute_name)
);

-- 데이터 삽입
INSERT INTO adhoc.product_attributes VALUES
    (1, 'weight', 0.5),           -- 무게 (kg)
    (1, 'color', 'red'),          -- 색상
    (1, 'in_stock', true),        -- 재고 여부
    (1, 'dimensions', '10x20x5'), -- 크기
    (2, 'weight', 0.3),
    (2, 'color', 'blue'),
    (2, 'in_stock', false),
    (2, 'max_temperature', 100);  -- 최대 온도 (섭씨)

In [ ]:
%%sql

-- 데이터 조회
SELECT * FROM adhoc.product_attributes;

In [ ]:
%%sql

-- 특정 제품의 모든 속성 조회
SELECT attribute_name, attribute_value
FROM adhoc.product_attributes
WHERE product_id = 1;

In [ ]:
%%sql

-- 재고가 있는 제품만 조회
SELECT DISTINCT product_id
FROM adhoc.product_attributes
WHERE attribute_name = 'in_stock' AND attribute_value.b = true;

In [ ]:
%%sql

-- 무게가 0.4kg 이상인 제품 조회
SELECT product_id
FROM adhoc.product_attributes
WHERE attribute_name = 'weight' AND attribute_value.f >= 0.4;

## GROUP BY & Aggregation Functions

1. 월별 세션수를 계산하는 SQL

In [ ]:
%%sql

SELECT
    LEFT(ts::varchar, 7) AS mon,
    COUNT(1) AS session_count
FROM raw_data.session_timestamp
GROUP BY 1
ORDER BY 1;

2. 가장 많이 사용된 채널은 무엇인가?

In [ ]:
%%sql

SELECT
    channel,
    COUNT(1) AS session_count,
    COUNT(DISTINCT userId) AS user_count
FROM raw_data.user_session_channel
GROUP BY 1                         -- GROUP BY channel
ORDER BY 2 DESC;              -- ORDER BY session_count DESC

In [ ]:
%%sql

SELECT
    channel,
    COUNT(1) AS session_count,
    COUNT(DISTINCT userId) AS user_count
FROM raw_data.user_session_channel
GROUP BY 1                         -- GROUP BY channel
ORDER BY 2 DESC              -- ORDER BY session_count DESC
LIMIT 1;

In [ ]:
%%sql

SELECT
    channel,
    COUNT(1) AS session_count,
    COUNT(DISTINCT sessionId) AS session_count2,
    COUNT(DISTINCT userId) AS user_count
FROM raw_data.user_session_channel
GROUP BY channel
ORDER BY session_count DESC;

In [ ]:
%%sql

SELECT
    channel,
    COUNT(1) AS session_count,
    COUNT(DISTINCT userId) AS user_count
FROM raw_data.user_session_channel
GROUP BY 1                         -- GROUP BY channel
ORDER BY 3 DESC                    -- ORDER BY user_count DESC
LIMIT 1;

3. 가장 많은 세션을 만들어낸 사용자 ID는 무엇인가?

In [ ]:
%%sql

SELECT
    userId,
    COUNT(1) AS count
FROM raw_data.user_session_channel
GROUP BY 1                         -- GROUP BY userId
ORDER BY 2 DESC              -- ORDER BY count DESC
LIMIT 1;

4. 월별 유니크한 사용자 수 (MAU: Monthly Active User)

In [ ]:
%%sql

SELECT
  STRFTIME(A.ts, '%Y-%m') AS year_month,
  COUNT(DISTINCT B.userid) AS mau
FROM raw_data.session_timestamp A
JOIN raw_data.user_session_channel B ON A.sessionid = B.sessionid
GROUP BY 1
ORDER BY 1 DESC;

5. 월별 채널별 유니크한 사용자 수

In [ ]:
%%sql

SELECT
  STRFTIME(A.ts, '%Y-%m') AS year_month,
  channel,
  COUNT(DISTINCT B.userid) AS mau
FROM raw_data.session_timestamp A
JOIN raw_data.user_session_channel B ON A.sessionid = B.sessionid
GROUP BY 1, 2
ORDER BY 1 DESC, 2;

## JOIN

In [ ]:
%%sql

DROP TABLE IF EXISTS raw_data.vital;
CREATE TABLE raw_data.vital (
    UserID int,
    VitalID	int PRIMARY KEY,
    Date date,
    Weight int
)
;
INSERT INTO raw_data.vital VALUES
(100,	1, '2020-01-01', 75),
(100, 3, '2020-01-02', 78),
(101, 2, '2020-01-01', 90),
(101, 4, '2020-01-02', 95);

In [ ]:
%%sql

DROP TABLE IF EXISTS raw_data.alert;
CREATE TABLE raw_data.alert (
    AlertID int PRIMARY KEY,
    VitalID	int,
    AlertType varchar(32),
    Date date,
    UserID int
);
INSERT INTO raw_data.alert VALUES
 (1,	4, 'WeightIncrease', '2020-01-01', 101),
 (2, NULL, 'MissingVital', '2020-01-04', 100),
 (3, NULL, 'MissingVital', '2020-01-04', 101);

In [ ]:
%%sql

SELECT *
FROM raw_data.vital

In [ ]:
%%sql

SELECT *
FROM raw_data.alert

* INNER JOIN
1. 양쪽 테이블에서 매치가 되는 레코드들만 리턴함
2. 양쪽 테이블의 필드가 모두 채워진 상태로 리턴됨

In [ ]:
%%sql

SELECT *
FROM raw_data.vital v
JOIN raw_data.alert a ON v.vitalid = a.vitalid;

* LEFT JOIN: 왼쪽 테이블의 레코드는 모두 리턴되며 오른쪽 테이블과 매칭이 되는 레코드들의 경우에는 오른쪽 테이블 레코드들의 컬럼들이 채워진 상태로 리턴됨. 매칭이 안되는 왼쪽 테이블 레코드들은 오른쪽 테이블에서 들어오는 필드들은 NULL로 채워짐

In [ ]:
%%sql

SELECT *
FROM raw_data.vital v
LEFT JOIN raw_data.alert a ON v.vitalid = a.vitalid;

In [ ]:
%%sql

SELECT *
FROM raw_data.vital v
RIGHT JOIN raw_data.alert a ON v.vitalid = a.vitalid;

* FULL JOIN (OUTER JOIN):
1. 왼쪽 테이블과 오른쪽 테이블의 모든 레코드들을 리턴함
2. 매칭되는 경우에만 양쪽 테이블들의 모든 필드들이 채워진 상태로 리턴됨

In [ ]:
%%sql

SELECT *
FROM raw_data.vital v
FULL JOIN raw_data.alert a ON v.vitalid = a.vitalid;

* CARTESIAN JOIN (CROSS JOIN): 조인 조건 없이 두 개 테이블의 내용을 모두 조합한 결과 레코드들을 생성

In [ ]:
%%sql

SELECT * FROM raw_data.vital v
CROSS JOIN raw_data.alert a;

SELF JOIN

In [ ]:
%%sql

SELECT * FROM raw_data.vital v1
JOIN raw_data.vital v2 ON v1.vitalid = v2.vitalid;

## CTAS, CTE, Subquery

In [ ]:
%%sql

DROP TABLE IF EXISTS analytics.session_summary;
CREATE TABLE analytics.session_summary AS
SELECT B.*, A.ts
FROM raw_data.session_timestamp A
JOIN raw_data.user_session_channel B ON A.sessionid = B.sessionid;

In [ ]:
%%sql

SELECT
  STRFTIME(ts, '%Y-%m') AS year_month,
  COUNT(DISTINCT userid) AS mau
FROM analytics.session_summary
GROUP BY 1
ORDER BY 1 DESC;

In [ ]:
%%sql

-- CTE 사용
WITH tmp AS (
    SELECT B.*, A.ts
    FROM raw_data.session_timestamp A
    JOIN raw_data.user_session_channel B ON A.sessionid = B.sessionid
)
SELECT
  STRFTIME(ts, '%Y-%m') AS year_month,
  COUNT(DISTINCT userid) AS mau
FROM tmp
GROUP BY 1
ORDER BY 1 DESC;

In [ ]:
%%sql

-- Subquery 사용
SELECT
  STRFTIME(ts, '%Y-%m') AS year_month,
  COUNT(DISTINCT userid) AS mau
FROM (
    SELECT B.*, A.ts
    FROM raw_data.session_timestamp A
    JOIN raw_data.user_session_channel B ON A.sessionid = B.sessionid
)
GROUP BY 1
ORDER BY 1 DESC;

## 데이터 품질 체크 예 보기

중복된 레코드들 체크하기

In [ ]:
%%sql

SELECT COUNT(1)
FROM analytics.session_summary

In [ ]:
%%sql

SELECT COUNT(1)
FROM (
    SELECT DISTINCT userId, sessionId, ts, channel
    FROM analytics.session_summary
);

In [ ]:
%%sql

With ds AS (
    SELECT DISTINCT userId, sessionId, ts, channel
    FROM analytics.session_summary
)
SELECT COUNT(1)
FROM ds;


최근 데이터의 존재 여부 체크하기 (freshness)

In [ ]:
%%sql

SELECT MIN(ts), MAX(ts)
FROM analytics.session_summary;

Primary key uniqueness가 지켜지는지 체크하기

In [ ]:
%%sql

SELECT sessionId, COUNT(1)
FROM analytics.session_summary
GROUP BY 1
ORDER BY 2 DESC
LIMIT 1;

In [ ]:
%%sql

SELECT
   COUNT(CASE WHEN sessionId is NULL THEN 1 END) sessionid_null_count,
   COUNT(CASE WHEN userId is NULL THEN 1 END) userid_null_count,
   COUNT(CASE WHEN ts is NULL THEN 1 END) ts_null_count,
   COUNT(CASE WHEN channel is NULL THEN 1 END) channel_null_count
FROM analytics.session_summary;

## 기타 문법 학습 (COALESCE, NULLIF 등등)

BOOLEAN, NULL 비교

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.boolean_test;
CREATE TABLE adhoc.boolean_test (
    flag BOOLEAN
);
INSERT INTO adhoc.boolean_test VALUES (True), (False), (TRUE), (NULL), (false);

In [ ]:
%%sql

SELECT *
FROM adhoc.boolean_test;

In [ ]:
%%sql

SELECT
    COUNT(CASE WHEN flag = True THEN 1 END) true_cnt1,
    COUNT(CASE WHEN flag is True THEN 1 END) true_cnt2,
    COUNT(CASE WHEN flag is not False THEN 1 END) not_false_cnt
FROM adhoc.boolean_test;

In [ ]:
%%sql

SELECT COUNT(1)
FROM adhoc.boolean_test
WHERE flag is NULL;

In [ ]:
%%sql

SELECT COUNT(1)
FROM adhoc.boolean_test
WHERE flag = NULL;

공백이 들어있는 필드이름 혹은 예약된 키워드를 필드이름으로 사용하려면?

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.test;
CREATE TABLE adhoc.test (
    group int primary key,
    'mailing address' varchar(32)
);

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.test;
CREATE TABLE adhoc.test (
    group int primary key,
    "mailing address" varchar(32)
);

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.test;
CREATE TABLE adhoc.test (
    "group" int primary key,
    "mailing address" varchar(32)
);

In [ ]:
%%sql

SELECT "mailing address", "group"
FROM adhoc.test;

COALESCE와 NULLIF 학습

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.count_test;
CREATE TABLE adhoc.count_test (
  value int
);
INSERT INTO adhoc.count_test VALUES (NULL),(1), (1), (0), (0), (4), (3);

In [ ]:
%%sql

SELECT
     value,
     COALESCE(value, 0)
FROM adhoc.count_test;

In [ ]:
%%sql

SELECT 100./0

In [ ]:
%%sql

SELECT
     value,
     100/value
FROM adhoc.count_test;

In [ ]:
%%sql

SELECT
     value, NULLIF(value, 0), 100/NULLIF(value, 0)
FROM adhoc.count_test;

ROW_NUMBER 실습

In [ ]:
%%sql

SELECT userid, ts, channel,
    ROW_NUMBER() OVER(PARTITION BY userid ORDER BY ts) forward_seq,
    ROW_NUMBER() OVER(PARTITION BY userid ORDER BY ts DESC) backward_seq
FROM raw_data.user_session_channel usc
LEFT JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid;

In [ ]:
%%sql

DROP TABLE IF EXISTS raw_data.rows_test;
CREATE TABLE raw_data.rows_test (
  value int
);
INSERT INTO raw_data.rows_test VALUES (1), (2), (3), (4), (5);

In [ ]:
%%sql

SELECT
    value,
    SUM(value) OVER (order by value rows between 2 preceding and 2 following) AS rolling_sum
FROM raw_data.rows_test ;

In [ ]:
%%sql

SELECT
    value,
    FIRST_VALUE(value) OVER (order by value rows between 2 preceding and 2 following) AS first_value,
    LAST_VALUE(value) OVER (order by value rows between 2 preceding and 2 following) AS last_value
FROM raw_data.rows_test ;

# 문제 풀이

## 1. 월별 채널별 매출액 및 상세 정보 테이블 만들기

### 어떤 형태의 조인을 해야하는지 확인

In [ ]:
%%sql

SELECT DISTINCT sessionid FROM raw_data.session_timestamp
EXCEPT
SELECT DISTINCT sessionid FROM raw_data.user_session_channel
;

In [ ]:
%%sql

SELECT DISTINCT sessionid FROM raw_data.user_session_channel
EXCEPT
SELECT DISTINCT sessionid FROM raw_data.session_timestamp
;

In [ ]:
%%sql

SELECT COUNT(1)
FROM (
  SELECT DISTINCT sessionid FROM raw_data.session_timestamp
  EXCEPT
  SELECT DISTINCT sessionid FROM raw_data.session_transaction
)

In [ ]:
%%sql

SELECT COUNT(1)
FROM (
  SELECT DISTINCT sessionid FROM raw_data.session_transaction
  EXCEPT
  SELECT DISTINCT sessionid FROM raw_data.session_timestamp
)

In [ ]:
%%sql

SELECT * FROM raw_data.session_transaction
WHERE amount < 0

### Summary Table 만들기

In [ ]:
%%sql

SELECT LEFT(ts::varchar, 7) "month",  -- "year month"
       channel,
       COUNT(DISTINCT usc.userid) uniqueUsers,
       COUNT(DISTINCT CASE WHEN amount > 0 THEN usc.userid END) paidUsers,
       ROUND(paidUsers*100.0/NULLIF(uniqueUsers, 0),2) conversionRate,
       SUM(amount) grossRevenue,
       SUM(CASE WHEN refunded is False THEN amount END) netRevenue
   FROM raw_data.user_session_channel usc
   LEFT JOIN raw_data.session_timestamp t ON t.sessionid = usc.sessionid
   LEFT JOIN raw_data.session_transaction st ON st.sessionid = usc.sessionid
   GROUP BY 1, 2
   ORDER BY 1, 2;

In [ ]:
%%sql

--혹시 기존에 생성되어 있으면 삭제
DROP TABLE IF EXISTS analytics.monthly_channel_summary;

--Summary Table 생성
CREATE TABLE analytics.monthly_channel_summary
AS
SELECT LEFT(ts::varchar, 7) year_month,
    usc.channel,
    COUNT(DISTINCT usc.userid) unique_users,
    COUNT(DISTINCT CASE WHEN amount>0 THEN userid END) paid_users,
    ROUND(paid_users*100./NULLIF(unique_users,0),2) conversion_rate,
    SUM(amount) gross_revenue,
    SUM(CASE WHEN refunded is False THEN amount
        ELSE 0 END) net_revenue
  FROM raw_data.user_session_channel usc
  LEFT JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
  LEFT JOIN raw_data.session_transaction str ON usc.sessionid = str.sessionid
GROUP BY 1, 2;

In [ ]:
%%sql

--정상적으로 생성되었는지 확인
SELECT * FROM analytics.monthly_channel_summary;

## 2. 사용자별로 처음 채널과 마지막 채널 알아내기

In [ ]:
%%sql

WITH first AS (
   SELECT userid, ts, channel, ROW_NUMBER() OVER(PARTITION BY userid ORDER BY ts) seq
   FROM raw_data.user_session_channel usc
   JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
), last AS (
   SELECT userid, ts, channel, ROW_NUMBER() OVER(PARTITION BY userid ORDER BY ts DESC) seq
   FROM raw_data.user_session_channel usc
   JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
)
SELECT first.userid AS userid, first.channel AS first_channel, last.channel AS last_channel
FROM first
JOIN last ON first.userid = last.userid and last.seq = 1
WHERE first.seq = 1
ORDER BY 1;

In [ ]:
%%sql

SELECT first.userid AS userid, first.channel AS first_channel, last.channel AS last_channel
FROM (
  SELECT userid, ts, channel, ROW_NUMBER() OVER(PARTITION BY userid ORDER BY ts) seq
  FROM raw_data.user_session_channel usc
  JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
) first
JOIN (
  SELECT userid, ts, channel, ROW_NUMBER() OVER(PARTITION BY userid ORDER BY ts DESC) seq
  FROM raw_data.user_session_channel usc
  JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
)  last ON first.userid = last.userid and last.seq = 1
WHERE first.seq = 1
ORDER BY userid;

In [ ]:
%%sql

SELECT *
FROM (
  SELECT
    userid,
    channel,
    (ROW_NUMBER() OVER (PARTITION BY usc.userid ORDER BY  st.ts asc)) AS rn1,
    (ROW_NUMBER() OVER (PARTITION BY usc.userid ORDER BY  st.ts desc)) AS rn2
  FROM raw_data.user_session_channel usc
  JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
)
WHERE rn1 = 1 OR rn2 = 1
ORDER BY userid, rn1, rn2;


In [ ]:
%%sql

SELECT userid,
 MAX(CASE WHEN rn1 = 1 THEN channel END) first_touch,
 MAX(CASE WHEN rn2 = 1 THEN channel END) last_touch
FROM (
  SELECT userid,
    channel,
    (ROW_NUMBER() OVER (PARTITION BY usc.userid ORDER BY  st.ts asc)) AS rn1,
    (ROW_NUMBER() OVER (PARTITION BY usc.userid ORDER BY  st.ts desc)) AS rn2
  FROM raw_data.user_session_channel usc
  JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
)
GROUP BY 1
ORDER BY 1;

In [ ]:
%%sql

SELECT
    A.userid,
    FIRST_VALUE(A.channel) over(partition by A.userid order by B.ts
rows between unbounded preceding and unbounded following) AS First_Channel,
    LAST_VALUE(A.channel) over(partition by A.userid order by B.ts
rows between unbounded preceding and unbounded following) AS Last_Channel
FROM raw_data.user_session_channel A
LEFT JOIN raw_data.session_timestamp B ON A.sessionid = B.sessionid
ORDER BY 1;

In [ ]:
%%sql

SELECT DISTINCT
    A.userid,
    FIRST_VALUE(A.channel) over(partition by A.userid order by B.ts
rows between unbounded preceding and unbounded following) AS First_Channel,
    LAST_VALUE(A.channel) over(partition by A.userid order by B.ts
rows between unbounded preceding and unbounded following) AS Last_Channel
FROM raw_data.user_session_channel A
LEFT JOIN raw_data.session_timestamp B ON A.sessionid = B.sessionid
ORDER BY 1;

## 3. Gross Revenue가 가장 큰 UserID 10개


In [ ]:
%%sql

SELECT
    userID,
    SUM(amount) as gross_revenue
FROM raw_data.session_transaction st
LEFT JOIN raw_data.user_session_channel usc ON st.sessionid = usc.sessionid
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10;

In [ ]:
%%sql

SELECT
    usc.userid,
    amount,
    SUM(amount) OVER(PARTITION BY usc.userid) gross_revenue
FROM raw_data.user_session_channel AS usc
JOIN raw_data.session_transaction AS revenue ON revenue.sessionid = usc.sessionid
ORDER BY 3 DESC
LIMIT 10;

In [ ]:
%%sql

SELECT DISTINCT
    usc.userid,
    SUM(amount) OVER(PARTITION BY usc.userid) gross_revenue
FROM raw_data.user_session_channel AS usc
JOIN raw_data.session_transaction AS revenue ON revenue.sessionid = usc.sessionid
ORDER BY 2 DESC
LIMIT 10;

## 4. nps 테이블을 바탕으로 월별 NPS 계산



In [ ]:
%%sql

DROP TABLE IF EXISTS raw_data.nps;
CREATE TABLE raw_data.nps
(
    id int PRIMARY KEY,
    created TIMESTAMP NOT NULL,
    score SMALLINT NOT NULL
);

In [ ]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/nps.csv

In [ ]:
%%sql

COPY raw_data.nps FROM 'nps.csv' (DELIMITER ',');

In [ ]:
%%sql

SELECT month,
  ROUND((promoters-detractors)::float*100/total_count, 2) AS overall_nps
FROM (
  SELECT LEFT(created::varchar, 7) AS month,
    COUNT(CASE WHEN score >= 9 THEN 1 END) AS promoters,
    COUNT(CASE WHEN score <= 6 THEN 1 END) AS detractors,
    COUNT(CASE WHEN score > 6 AND score < 9 THEN 1 END) As passives,
    COUNT(1) AS total_count
  FROM raw_data.nps
  GROUP BY 1
  ORDER BY 1
);

In [ ]:
%%sql

SELECT LEFT(created::varchar, 7) AS month,
  ROUND(SUM(CASE
    WHEN score >= 9 THEN 1
    WHEN score <= 6 THEN -1 END)::float*100/COUNT(1), 2) overall_nps
FROM raw_data.nps
GROUP BY 1
ORDER BY 1;

## 5. 가상 이커머스 사이트의 인기 상품 찾기 (주문 수량/금액 기준)


In [ ]:
%%sql

DROP TABLE IF EXISTS raw_data.order;
CREATE TABLE raw_data.order (
    order_id character varying(32) PRIMARY KEY,
    ordered_at timestamp,
    refunded_at timestamp
);

DROP TABLE IF EXISTS raw_data.order_item;
CREATE TABLE raw_data.order_item (
    order_item_id int PRIMARY KEY,
    order_id character varying(32),
    product_name character varying(32),
    quantity smallint,
    amount float
);

In [ ]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/order.csv
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/order_item.csv

In [ ]:
%%sql

COPY raw_data.order FROM 'order.csv' (DELIMITER ',');
COPY raw_data.order_item FROM 'order_item.csv' (DELIMITER ',');

환불되지 않은 주문들만 봤을 때 주문수량 (quantity) 기준으로 가장 많이 주문된 Top 5 상품의 이름과 총 주문수량을 찾아보세요

In [ ]:
%%sql

SELECT oi.product_name, SUM(quantity) total_quantity
FROM raw_data.order o
JOIN raw_data.order_item oi ON o.order_id = oi.order_id
WHERE o.refunded_at IS NULL
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;

In [ ]:
%%sql

-- 환불 포함해서 계산
SELECT oi.product_name, SUM(quantity) total_quantity
FROM raw_data.order o
JOIN raw_data.order_item oi ON o.order_id = oi.order_id
-- WHERE o.refunded_at IS NULL
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;

환불되지 않은 주문들만 봤을 때 주문금액 (amount*quantity) 기준으로 가장 많이 주문된 Top 5 상품의 이름과 총 주문금액을 찾아보세요.

In [ ]:
%%sql

SELECT oi.product_name, SUM(quantity*amount)::decimal(7,2) total_amount
FROM raw_data.order o
JOIN raw_data.order_item oi ON o.order_id = oi.order_id
WHERE o.refunded_at IS NULL
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;

In [ ]:
%%sql

-- 환불 포함해서 계산
SELECT oi.product_name, SUM(quantity*amount)::decimal(7,2) total_amount
FROM raw_data.order o
JOIN raw_data.order_item oi ON o.order_id = oi.order_id
-- WHERE o.refunded_at IS NULL
GROUP BY 1
ORDER BY 2 DESC
LIMIT 5;

## 트랜잭션 학습

- Transaction 자체는 Python API로만 진행 가능. Python Notebook에서는 적어도 DuckDB transaction을 사용할 수 없음 (사용시 에러 발생)

In [ ]:
import duckdb

duckdb_con = duckdb.connect("duckdb.db")

In [ ]:
%%sql

DROP TABLE IF EXISTS adhoc.name_gender;
CREATE TABLE adhoc.name_gender (
  name varchar(32) primary key,
  gender varchar(16)
);

In [ ]:
%%sql

INSERT INTO adhoc.name_gender VALUES ('Ben', 'Male'), ('Maddie', 'Female');

In [ ]:
duckdb_con.execute("SELECT * FROM adhoc.name_gender;").df()

In [ ]:
duckdb_con.execute("BEGIN;")
duckdb_con.execute("DELETE FROM adhoc.name_gender;")
# adhoc.name_gender 테이블에 새로 레코드 적재
duckdb_con.execute("ROLLBACK;")

In [ ]:
duckdb_con.execute("SELECT * FROM adhoc.name_gender;").df()

In [ ]:
duckdb_con.execute("BEGIN;")
duckdb_con.execute("DELETE FROM adhoc.name_gender;")
# adhoc.name_gender 테이블에 새로 레코드 적재
duckdb_con.execute("COMMIT;")

In [ ]:
duckdb_con.execute("SELECT * FROM adhoc.name_gender;").df()

CTAS로 ELT를 구현하는 경우

In [ ]:
%%sql

DROP TABLE IF EXISTS analytics.channel;
CREATE TABLE analytics.channel AS SELECT DISTINCT channel FROM raw_data.user_session_channel;

In [ ]:
duckdb_con.execute("DROP TABLE IF EXISTS analytics.temp_channel;")
duckdb_con.execute("""
CREATE TABLE analytics.temp_channel AS
SELECT DISTINCT channel FROM raw_data.user_session_channel;""")
# temp_channel에 대해 테스트 수행
# 최종 transaction 수행
duckdb_con.execute("BEGIN;")
duckdb_con.execute("DROP TABLE IF EXISTS analytics.channel;")
duckdb_con.execute("ALTER TABLE analytics.temp_channel RENAME TO channel;")
duckdb_con.execute("END;")

In [ ]:
%%sql

SELECT * FROM analytics.channel;

데이터 파이프라인에서 전체 업데이트(Full Refresh)를 하는 경우


In [ ]:
try:
  duckdb_con.execute("BEGIN")

  duckdb_con.execute("DROP TABLE IF EXISTS raw_data.user_balance")
  duckdb_con.execute("CREATE TABLE raw_data.user_balance ( userId int PRIMARY KEY, amount float )")
  # 어디에선가 읽어온 정보를 raw_data.user_balance 테이블에 적재
  duckdb_con.execute("INSERT INTO raw_data.user_balance VALUES (1, 1000), (2, 2000), (3, 3000)")

  # Commit!
  duckdb_con.commit()

except Exception as e:
  duckdb_con.rollback()
  print("Error occurred:" + str(e))

In [ ]:
duckdb_con.execute("""
SELECT *
FROM raw_data.user_balance;""").df()

In [ ]:
try:
  duckdb_con.execute("BEGIN")

  duckdb_con.execute("DROP TABLE IF EXISTS raw_data.user_balance")
  duckdb_con.execute("CREATE TABLE raw_data.user_balance ( userId int PRIMARY KEY, amount float )")
  # 어디에선가 읽어온 정보를 raw_data.user_balance 테이블에 적재
  duckdb_con.execute("INSERT INTO raw_data.user_balance VALUES (1, 1000), (1, 2000), (3, 3000)")

  # Commit!
  duckdb_con.commit()

except Exception as e:
  duckdb_con.rollback()
  print("Error occurred:" + str(e))

In [ ]:
duckdb_con.execute("""
SELECT *
FROM raw_data.user_balance;""").df()

## 기타 고급 문법 소개

In [ ]:
%%sql

SELECT 'mark' as first_name, 'zuckerberg' as last_name

UNION

SELECT 'elon', 'musk'

UNION

SELECT 'mark', 'zuckerberg'

In [ ]:
%%sql

SELECT 'mark' as first_name, 'zuckerberg' as last_name

UNION ALL

SELECT 'elon', 'musk'

UNION ALL

SELECT 'mark', 'zuckerberg'

INTERSECT: SELECT 문들에서 공통적인 레코드를 돌려줌 (교집합)

In [ ]:
%%sql

SELECT 'mark' as first_name, 'zuckerberg' as last_name

INTERSECT

SELECT 'mark', 'zuckerberg'

In [ ]:
%%sql

SELECT 'mark' as first_name, 'zuckerberg' as last_name

EXCEPT

SELECT 'mark', 'zuckerberg'

In [ ]:
%%sql

SELECT COUNT(1)
FROM (
  SELECT sessionId FROM raw_data.user_session_channel
  INTERSECT
  SELECT sessionId FROM raw_data.session_transaction
);

In [ ]:
%%sql

SELECT COUNT(1)
FROM (
  SELECT sessionId FROM raw_data.user_session_channel
  EXCEPT
  SELECT sessionId FROM raw_data.session_transaction
);

STRING_AGG - GROUP BY Aggregate 함수 중의 하나

In [ ]:
%%sql

SELECT userid, STRING_AGG(channel, ',' ORDER BY ts) channels
FROM raw_data.user_session_channel usc
JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
GROUP BY 1
LIMIT 10;

In [ ]:
%%sql

SELECT usc.*, st.ts, LAG(channel, 1) OVER (PARTITION BY userId ORDER BY ts) prev_channel
FROM raw_data.user_session_channel usc
JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
ORDER BY usc.userid, st.ts
LIMIT 100;

In [ ]:
%%sql

SELECT usc.*, st.ts, LAG(channel, 1) OVER (PARTITION BY userId ORDER BY ts DESC) next_channel
FROM raw_data.user_session_channel usc
JOIN raw_data.session_timestamp st ON usc.sessionid = st.sessionid
ORDER BY usc.userid, st.ts
LIMIT 100;

JSON parsing 함수

In [ ]:
%%sql

SELECT JSON_EXTRACT(
  '{"name": "Alice", "age": 30, "address": {"city": "Wonderland", "zip": "12345"}}',
  'address'
) AS address;

In [ ]:
%%sql

-- $ (Root Element)
SELECT json_extract(
  '{"name": "Alice", "age": 30, "address": {"city": "Wonderland", "zip": "12345"}}',
  '$.address.city'
)AS city;

## AI 활용하기

도움 #1:  월별 채널별 매출 상세정보 - 프롬프트 작성

아래 세 테이블이 있어. 세 테이블은 sessionId로 조인이 가능하고 channel이 세션생성시 채널 정보이고 ts는 세션생성 시간이야. session_transaction 테이블 같은 경우 매출 세션에 관한 정보인데 amount가 매출액이고 refunded는 true인 경우 환불이 된 거야.

<pre>
CREATE TABLE session_timestamp (
    ts TIMESTAMP,
    sessionId VARCHAR PRIMARY KEY
);
CREATE TABLE user_session_channel (
    userId INT,
    sessionId varchar PRIMARY KEY,
    channel VARCHAR
);
CREATE TABLE session_transaction (
    sessionId VARCHAR PRIMARY KEY,
    amount int,
    refunded boolean
);
</pre>

위 세 테이블을 조인해서 아래 정보를 계산하고 싶어.

 * month: 세션 발생 연도와 월
 * channel: 세션 발생 채널
 * uniqueUsers: 총방문 사용자
 * paidUsers: 구매 사용자. refund한 경우도 판매로 고려해줘
 * conversionRate: 구매사용자 / 총방문 사용자
 * grossRevenue: 환불 포함한 매출액 (amount)
 * netRevenue: 환불 제외한 매출액 (amount)

DuckDB 기준으로 위 SQL을 작성해줘. SQL 전문가인 너를 믿고 맡기는 거니 잘 해줘.

ChatGPT

In [ ]:
%%sql

SELECT
    DATE_TRUNC('month', st.ts) AS month,
    usc.channel AS channel,
    COUNT(DISTINCT usc.userId) AS uniqueUsers,
    COUNT(DISTINCT CASE WHEN stx.amount > 0 THEN usc.userId END) AS paidUsers,
    ROUND(COUNT(DISTINCT CASE WHEN stx.amount > 0 THEN usc.userId END) * 100.0 / COUNT(DISTINCT usc.userId), 2) AS conversionRate,
    SUM(CASE WHEN stx.refunded = FALSE THEN stx.amount ELSE 0 END) AS grossRevenue,
    SUM(stx.amount) AS netRevenue
FROM
    raw_data.session_timestamp st
JOIN
    raw_data.user_session_channel usc ON st.sessionId = usc.sessionId
LEFT JOIN
    raw_data.session_transaction stx ON st.sessionId = stx.sessionId
GROUP BY
    month, channel
ORDER BY
    month, channel;

Claude

In [ ]:
%%sql

WITH joined_data AS (
  SELECT
    DATE_TRUNC('month', st.ts) AS month,
    usc.channel,
    usc.userId,
    COALESCE(str.amount, 0) AS amount,
    CASE WHEN str.refunded = true THEN 0 ELSE COALESCE(str.amount, 0) END AS net_amount
  FROM raw_data.session_timestamp st
  LEFT JOIN raw_data.user_session_channel usc ON st.sessionId = usc.sessionId
  LEFT JOIN raw_data.session_transaction str ON st.sessionId = str.sessionId
)
SELECT
  month,
  channel,
  COUNT(DISTINCT userId) AS uniqueUsers,
  COUNT(DISTINCT CASE WHEN amount > 0 THEN userId END) AS paidUsers,
  ROUND(CAST(COUNT(DISTINCT CASE WHEN amount > 0 THEN userId END) AS FLOAT) /
        NULLIF(COUNT(DISTINCT userId), 0), 4) AS conversionRate,
  SUM(amount) AS grossRevenue,
  SUM(net_amount) AS netRevenue
FROM joined_data
GROUP BY month, channel
ORDER BY month, channel;

Gemini

In [ ]:
%%sql

SELECT
  EXTRACT(YEAR FROM ts) AS year,
  EXTRACT(MONTH FROM ts) AS month,
  channel,
  COUNT(DISTINCT userId) AS uniqueUsers,
  COUNT(DISTINCT userId) AS paidUsers,
  SUM(amount) AS grossRevenue,
  SUM(CASE WHEN refunded THEN 0 ELSE amount END) AS netRevenue
FROM raw_data.session_timestamp st
JOIN raw_data.user_session_channel uc ON st.sessionId = uc.sessionId
JOIN raw_data.session_transaction st ON st.sessionId = st.sessionId
GROUP BY year, month, channel

도움 #2:  사용자별 첫 채널과 마지막 채널 - 프롬프트 작성

아래 두 테이블이 있어. 두 테이블은 sessionId로 조인이 가능하고 channel이 세션생성시 채널 정보이고 ts는 세션생성 시간이야.

<pre>
CREATE TABLE session_timestamp (
    ts TIMESTAMP,
    sessionId VARCHAR PRIMARY KEY
);
CREATE TABLE user_session_channel (
    userId INT,
    sessionId varchar PRIMARY KEY,
    channel VARCHAR
);
</pre>

위 두 테이블을 조인해서 아래 정보를 계산하고 싶어.

 * userId: 각 사용자 ID
 * first_channel: 해당 사용자의 첫 번째 채널 (“ts” 기준)
 * last_channel: 해당 사용자의 마지막 채널 (“ts” 기준)

DuckDB 기준으로 위 SQL을 작성해줘. SQL 전문가인 너를 믿고 맡기는 거니 잘 해줘.

ChatGPT

In [ ]:
%%sql

WITH first_last_channel AS (
    SELECT
        usc.userId,
        FIRST_VALUE(usc.channel) OVER (PARTITION BY usc.userId ORDER BY st.ts) AS first_channel,
        LAST_VALUE(usc.channel) OVER (PARTITION BY usc.userId ORDER BY st.ts ROWS BETWEEN UNBOUNDED PRECEDING AND UNBOUNDED FOLLOWING) AS last_channel
    FROM
        raw_data.session_timestamp st
    JOIN
        raw_data.user_session_channel usc ON st.sessionId = usc.sessionId
)

SELECT
    userId,
    first_channel,
    last_channel
FROM
    first_last_channel
GROUP BY
    userId, first_channel, last_channel
ORDER BY
    userId;


Claude

In [ ]:
%%sql

WITH ranked_sessions AS (
  SELECT
    usc.userId,
    usc.channel,
    st.ts,
    ROW_NUMBER() OVER (PARTITION BY usc.userId ORDER BY st.ts ASC) AS first_rank,
    ROW_NUMBER() OVER (PARTITION BY usc.userId ORDER BY st.ts DESC) AS last_rank
  FROM raw_data.user_session_channel usc
  JOIN raw_data.session_timestamp st ON usc.sessionId = st.sessionId
)
SELECT
  userId,
  MAX(CASE WHEN first_rank = 1 THEN channel END) AS first_channel,
  MAX(CASE WHEN last_rank = 1 THEN channel END) AS last_channel
FROM ranked_sessions
WHERE first_rank = 1 OR last_rank = 1
GROUP BY userId
ORDER BY userId;

In [ ]:
%%sql

WITH ranked_sessions AS (
  SELECT
    usc.userId,
    usc.channel,
    st.ts,
    ROW_NUMBER() OVER (PARTITION BY usc.userId ORDER BY st.ts ASC) AS first_rank,
    ROW_NUMBER() OVER (PARTITION BY usc.userId ORDER BY st.ts DESC) AS last_rank
  FROM raw_data.user_session_channel usc
  JOIN raw_data.session_timestamp st ON usc.sessionId = st.sessionId
)
SELECT
  userId,
  MAX(CASE WHEN first_rank = 1 THEN channel END) AS first_channel,
  MAX(CASE WHEN last_rank = 1 THEN channel END) AS last_channel
FROM ranked_sessions
WHERE first_rank = 1 OR last_rank = 1
GROUP BY userId
ORDER BY userId;

Gemini

In [ ]:
%%sql

WITH first_channel AS (
  SELECT
    userId,
    channel,
    ROW_NUMBER() OVER (PARTITION BY userId ORDER BY ts ASC) AS row_number
  FROM raw_data.session_timestamp st
  JOIN raw_data.user_session_channel uc ON st.sessionId = uc.sessionId
),
last_channel AS (
  SELECT
    userId,
    channel,
    ROW_NUMBER() OVER (PARTITION BY userId ORDER BY ts DESC) AS row_number
  FROM raw_data.session_timestamp st
  JOIN raw_data.user_session_channel uc ON st.sessionId = uc.sessionId
)
SELECT
  fc.userId,
  fc.channel AS first_channel,
  lc.channel AS last_channel
FROM first_channel fc
JOIN last_channel lc ON fc.userId = lc.userId AND fc.row_number = 1 AND lc.row_number = 1
ORDER BY 1;

In [ ]:
%%sql

WITH first_channel AS (
  SELECT
    userId,
    channel,
    ROW_NUMBER() OVER (PARTITION BY userId ORDER BY ts ASC) AS row_number
  FROM raw_data.session_timestamp st
  JOIN raw_data.user_session_channel uc ON st.sessionId = uc.sessionId
),
last_channel AS (
  SELECT
    userId,
    channel,
    ROW_NUMBER() OVER (PARTITION BY userId ORDER BY ts DESC) AS row_number
  FROM raw_data.session_timestamp st
  JOIN raw_data.user_session_channel uc ON st.sessionId = uc.sessionId
)
SELECT
  fc.userId,
  fc.channel AS first_channel,
  lc.channel AS last_channel
FROM first_channel fc
JOIN last_channel lc ON fc.userId = lc.userId AND fc.row_number = 1 and lc.row_number = 1
ORDER BY 1;